In [ ]:
import json
from sentence_transformers import SentenceTransformer
import re

with open("agenda_final_file.json", 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

def transform(data):
    
    final_data = process_and_save_embeddings(extract_stakeholders(data), model_name='sentence-transformers/LaBSE')
    
def extract_stakeholders(data):
    
    data_with_names = []
    
    pattern1 = r'Question de (.*?)\s+\('
    pattern2 = r'Proposition de loi \((.*?)\)'
    pattern3 = r'Interpellation de (.*?) [à \ a]' 
    pattern4 = r'Proposition de résolution \((.*?)\)'

    for item in data:
        if 'items' in item:
            for para_item in item['items']:
                fr_text = para_item['fr_text']

                match_name = re.search(pattern1, fr_text)
                if match_name:
                    name = match_name.group(1)
                    para_item['Stakeholders'] = name

                matches_other = re.findall(pattern2, fr_text)
                if matches_other:
                    if 'Stakeholders' in para_item:
                        para_item['Stakeholders'] += ', ' + ', '.join(matches_other)
                    else:
                        para_item['Stakeholders'] = ', '.join(matches_other)

                match_pattern3 = re.search(pattern3, fr_text)
                if match_pattern3:
                    name_pattern3 = match_pattern3.group(1)
                    if 'Stakeholders' in para_item:
                        para_item['Stakeholders'] += ', ' + name_pattern3
                    else:
                        para_item['Stakeholders'] = name_pattern3

                # Check for pattern4 (Proposition de résolution) and add it to 'Stakeholders' field
                match_pattern4 = re.search(pattern4, fr_text)
                if match_pattern4:
                    name_pattern4 = match_pattern4.group(1)
                    if 'Stakeholders' in para_item:
                        para_item['Stakeholders'] += ', ' + name_pattern4
                    else:
                        para_item['Stakeholders'] = name_pattern4

                data_with_names.append(para_item)

    return data


def process_and_save_embeddings(data, model_name='sentence-transformers/LaBSE'):
     
    model = SentenceTransformer(model_name)

    
    for item in data:
        if 'items' in item:
            for para_item in item['items']:
                nl_text = para_item['nl_text'] 
                nl_text_embedding = model.encode(nl_text)
                para_item['nl_text_embedding'] = nl_text_embedding.tolist()
                fr_text = para_item['fr_text']
                fr_text_embedding = model.encode(fr_text)
                para_item['fr_text_embedding'] = fr_text_embedding.tolist()
    
    return data
